In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor

import keras
from keras.layers import Dense
from keras.models import Sequential
# from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import pickle

ModuleNotFoundError: No module named 'pandas'

# Load Data

In [ ]:

def loadData(fileName):
    dir=os.path.dirname(os.path.abspath("__file__"))
    filename = dir+'\\data\\' + fileName;
    print("filename--",filename)
    return pd.read_csv(filename, low_memory=False,encoding='latin-1')  

In [ ]:
flight_ds=loadData("final_data.csv")
flight_ds['SCH_DEP_DATE'] = pd.to_datetime(flight_ds['SCH_DEP_DATE']).dt.date
flight_ds['SCH_DEP_DAY'] = pd.to_datetime(flight_ds['SCH_DEP_DATE']).dt.day
flight_ds['SCH_DEP_MONTH'] = pd.to_datetime(flight_ds['SCH_DEP_DATE']).dt.month
flight_ds['SCH_DEP_YEAR'] = pd.to_datetime(flight_ds['SCH_DEP_DATE']).dt.year
flight_ds.head()


filename-- c:\DDrive\EGSAppWorkspace\dathathon\flightforcast\data\final_data.csv


,Unnamed: 0,Flight_No,org,dst,SCH_DEP_DATE,BKD_WT_sum,BKD_VOL_sum,CHG_WT_BU_sum,INHOUSE_AC_TYP,OPN_MODE,...,DST_AIRPORT_CODE,DST_AIRPORT_NAME,DST_CNTRY_NAME,DST_REGION_NAME,org_hday_flg,dst_hday_flg,EST_CHG_WT,SCH_DEP_DAY,SCH_DEP_MONTH,SCH_DEP_YEAR
0,0,6600.0,MUC,DWC,2018-01-19,13343.1,42.174486,13565.00,388L,J,...,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0,45293.924643,19,1,2018
1,1,6600.0,MUC,DWC,2019-02-10,8771.4,69.009161,12379.77,388R,J,...,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0,28980.045459,10,2,2019
2,2,6600.0,LIS,DWC,2018-07-24,14635.2,50.045059,15032.20,77WEJ,J,...,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0,17977.409472,24,7,2018
3,3,6600.0,LIS,DWC,2019-06-17,6509.6,37.940000,6509.60,77WER,J,...,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,0,27100.661797,17,6,2019
4,4,6600.0,LIS,DWC,2019-10-12,9936.0,52.440000,11709.34,77WER,J,...,DWC,Al Maktoum Intl,UAE,"GCC, MIDDLE EAST & IRAN",0,1,22324.243576,12,10,2019


In [ ]:
encode_columns = ['Flight_No', 'ISO_CODE', 'org', 'dst', 'INHOUSE_AC_TYP','OPN_MODE','SVC_TYP','SCH_DEP_DATE','ORG_REGION_NAME','DST_REGION_NAME','ORG_AIRPORT_CODE',	'ORG_AIRPORT_NAME','ORG_CNTRY_NAME','DST_AIRPORT_CODE','DST_AIRPORT_NAME','DST_CNTRY_NAME']
encode_df = flight_ds[encode_columns]
encode_df = encode_df.astype('str')
encode_df = encode_df.apply(LabelEncoder().fit_transform)
score_encode_drop = flight_ds.drop(encode_columns, axis = 1)
score_encode = pd.concat([score_encode_drop, encode_df], axis = 1)

# Train Test Split

In [ ]:
train, test = train_test_split(score_encode, test_size=0.20, random_state=42)
train.info()
display(train)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7205 entries, 5499 to 7270
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           7205 non-null   int64  
 1   BKD_WT_sum           7205 non-null   float64
 2   BKD_VOL_sum          7205 non-null   float64
 3   CHG_WT_BU_sum        7205 non-null   float64
 4   AC_CGO_WT_BU         7205 non-null   float64
 5   AC_CGO_VOL_BU        7205 non-null   float64
 6   CMRCL_CGO_WT_BU      7205 non-null   float64
 7   CMRCL_CGO_VOL_BU     7205 non-null   float64
 8   NOOP_CAN_IND         7205 non-null   int64  
 9   EST_CGO_PLOD_WT_BU   7205 non-null   float64
 10  EST_CGO_PLOD_VOL_BU  7205 non-null   float64
 11  org_hday_flg         7205 non-null   int64  
 12  dst_hday_flg         7205 non-null   int64  
 13  EST_CHG_WT           7205 non-null   float64
 14  SCH_DEP_DAY          7205 non-null   int64  
 15  SCH_DEP_MONTH        7205 non-null 

,Unnamed: 0,BKD_WT_sum,BKD_VOL_sum,CHG_WT_BU_sum,AC_CGO_WT_BU,AC_CGO_VOL_BU,CMRCL_CGO_WT_BU,CMRCL_CGO_VOL_BU,NOOP_CAN_IND,EST_CGO_PLOD_WT_BU,...,SVC_TYP,SCH_DEP_DATE,ORG_REGION_NAME,DST_REGION_NAME,ORG_AIRPORT_CODE,ORG_AIRPORT_NAME,ORG_CNTRY_NAME,DST_AIRPORT_CODE,DST_AIRPORT_NAME,DST_CNTRY_NAME
5499,5499,10379.90,52.516205,10678.360,15000.0,60.0,18196.48,75.455389,0,18196.48,...,4,384,5,2,5,1,16,9,6,9
5893,5893,414.00,2.482000,414.000,17000.0,90.0,17000.00,90.000000,0,17000.00,...,3,909,1,3,9,2,8,4,0,11
7909,7909,4317.00,51.500687,8444.514,13000.0,60.0,31923.00,73.970060,0,31923.00,...,4,1298,1,3,14,11,17,4,0,11
3514,3514,9232.50,64.474561,12533.000,7000.0,90.0,16590.97,120.710000,0,16590.97,...,4,110,1,3,14,11,17,4,0,11
5830,5830,20367.00,115.200000,21260.000,20000.0,100.0,542875.00,542.000000,0,41359.42,...,4,1129,0,3,12,7,5,4,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,5734,18638.00,80.000000,18638.000,13000.0,60.0,18638.00,80.000000,0,32135.79,...,4,787,0,3,0,3,1,4,0,11
5191,5191,30352.80,135.475945,32074.000,0.0,0.0,0.00,0.000000,0,0.00,...,0,1230,2,2,1,6,9,13,1,8
5390,5390,34643.83,107.356120,36629.430,20000.0,100.0,41000.00,135.000000,0,17000.00,...,3,853,5,2,5,1,16,9,6,9
860,860,11121.00,44.096107,11127.500,45000.0,250.0,45000.00,250.000000,0,17000.00,...,3,1123,5,2,5,1,16,6,10,12


In [ ]:
def create_features(df, label=None):
 df['Flight_No'] = df.Flight_No  
 df['ISO_CODE'] = df.ISO_CODE
 df['org'] = df.org
 df['dst'] = df.dst   
 df['SCH_DEP_DAY'] = df.SCH_DEP_DAY
 df['SCH_DEP_MONTH'] = df.SCH_DEP_MONTH
 df['SCH_DEP_YEAR'] = df.SCH_DEP_YEAR
 df['INHOUSE_AC_TYP'] = df.INHOUSE_AC_TYP
 df['OPN_MODE'] = df.OPN_MODE
 df['SVC_TYP'] = df.SVC_TYP
 df['EST_CHG_WT'] = df.EST_CHG_WT   
 X = df[['Flight_No','ISO_CODE','org','dst','SCH_DEP_DAY','SCH_DEP_MONTH','SCH_DEP_YEAR','INHOUSE_AC_TYP','OPN_MODE','SVC_TYP','EST_CHG_WT']]
 if label:
     y = df[label]
 return X, y
 return X


In [ ]:
train.shape
train.head()

,Unnamed: 0,BKD_WT_sum,BKD_VOL_sum,CHG_WT_BU_sum,AC_CGO_WT_BU,AC_CGO_VOL_BU,CMRCL_CGO_WT_BU,CMRCL_CGO_VOL_BU,NOOP_CAN_IND,EST_CGO_PLOD_WT_BU,...,SVC_TYP,SCH_DEP_DATE,ORG_REGION_NAME,DST_REGION_NAME,ORG_AIRPORT_CODE,ORG_AIRPORT_NAME,ORG_CNTRY_NAME,DST_AIRPORT_CODE,DST_AIRPORT_NAME,DST_CNTRY_NAME
5499,5499,10379.9,52.516205,10678.360,15000.0,60.0,18196.48,75.455389,0,18196.48,...,4,384,5,2,5,1,16,9,6,9
5893,5893,414.0,2.482000,414.000,17000.0,90.0,17000.00,90.000000,0,17000.00,...,3,909,1,3,9,2,8,4,0,11
7909,7909,4317.0,51.500687,8444.514,13000.0,60.0,31923.00,73.970060,0,31923.00,...,4,1298,1,3,14,11,17,4,0,11
3514,3514,9232.5,64.474561,12533.000,7000.0,90.0,16590.97,120.710000,0,16590.97,...,4,110,1,3,14,11,17,4,0,11
5830,5830,20367.0,115.200000,21260.000,20000.0,100.0,542875.00,542.000000,0,41359.42,...,4,1129,0,3,12,7,5,4,0,11


In [ ]:
test.shape

(1802, 33)

In [ ]:

def scale_data(train_set, test_set):
    #apply Min Max Scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train_set)
    
    # reshape training set
    # train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
    train_set_scaled = scaler.transform(train_set)
    
    # reshape test set
    # test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
    test_set_scaled = scaler.transform(test_set)
    
    X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1].ravel()
    X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1].ravel()
    
    return X_train, y_train, X_test, y_test, scaler

X_train, y_train, X_test, y_test, scaler_object = scale_data(train, test)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
y_train.shape

(230560,)

# Modeling Functions

In [ ]:
def undo_scaling(y_pred, x_test, scaler_obj, lstm=False):  
    #reshape y_pred
    y_pred = y_pred.reshape(y_pred.shape[0], 1, 1)
    
    if not lstm:
        x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])
    
    #rebuild test set for inverse transform
    pred_test_set = []
    for index in range(0,len(y_pred)):
        pred_test_set.append(np.concatenate([y_pred[index],x_test[index]],axis=1))
        
    #reshape pred_test_set
    pred_test_set = np.array(pred_test_set)
    pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
    
    #inverse transform
    pred_test_set_inverted = scaler_obj.inverse_transform(pred_test_set)
    
    return pred_test_set_inverted

In [ ]:
def predict_df(unscaled_predictions, original_df):
    #create dataframe that shows the predicted sales
    y_pred_slr= slr.predict(X_test)
    
    return y_pred_slr

In [ ]:
model_scores = {}

def get_scores(unscaled_df, original_df, model_name):
    lag=1802
    rmse = np.sqrt(mean_squared_error(original_df[-lag:], unscaled_df[-lag:]))
    mae = mean_absolute_error(original_df[-lag:], unscaled_df[-lag:])
    r2 = r2_score(original_df[-lag:], unscaled_df[-lag:])
    model_scores[model_name] = [rmse, mae, r2]

    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R2 Score: {r2}")

In [ ]:
def plot_results(results, original_df, model_name):

    fig, ax = plt.subplots(figsize=(15,5))
    sns.lineplot(original_df.date, original_df.sales, data=original_df, ax=ax, 
                 label='Original', color='mediumblue')
    sns.lineplot(results.date, results.pred_value, data=results, ax=ax, 
                 label='Predicted', color='Red')
    
    ax.set(xlabel = "Date",
           ylabel = "Sales",
           title = f"{model_name} Sales Forecasting Prediction")
    
    ax.legend()
    
    sns.despine()
    
    plt.savefig(f'../model_output/{model_name}_forecast.png')

In [ ]:
def run_model(train_data, test_data, model, model_name):
    
    X_train, y_train, X_test, y_test, scaler_object = scale_data(train_data, test_data)
    
    mod = model
    mod.fit(X_train, y_train)
    predictions = mod.predict(X_test)
    
    # Undo scaling to compare predictions against original data
    original_df=flight_ds
    unscaled = undo_scaling(predictions, X_test, scaler_object)
    # unscaled_df = predict_df(unscaled, original_df)
    
    slr = LinearRegression()  
    slr.fit(X_train, y_train)
    #Prediction of test set
    unscaled_df= slr.predict(X_test)
    
    #Predicted values
    print("Prediction2 for test set: {}".format(unscaled_df))
   
    slr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': unscaled_df} )
    slr_diff.head()
    display(slr_diff)
    get_scores(unscaled_df, y_test, model_name)
    # len(y_pred_slr)

    plt.scatter(X_test,y_test)
    plt.plot(X_test, unscaled_df, 'Red')
    plt.show()
    
    # plot_results(unscaled_df, original_df, model_name)

# Linear Regression

In [ ]:
run_model(train, test, LinearRegression(), 'LinearRegression')

ValueError: Found input variables with inconsistent numbers of samples: [7205, 230560]

# Random Forest Regressor

In [ ]:
# run_model(train, test, RandomForestRegressor(n_estimators=100, max_depth=20), 
#           'RandomForest')

# XGBoost

In [ ]:
# run_model(train, test, XGBRegressor( n_estimators=100, 
#                                     learning_rate=0.2, 
#                                     objective='reg:squarederror'), 'XGBoost')

# LSTM

In [ ]:
def lstm_model(train_data, test_data):
    
    X_train, y_train, X_test, y_test, scaler_object = scale_data(train_data, test_data)
    
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
   
    model = Sequential()
    model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), 
                   stateful=True))
    model.add(Dense(1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=200, batch_size=1, verbose=1, 
              shuffle=False)
    predictions = model.predict(X_test,batch_size=1)
    
    original_df = load_original_df()
    unscaled = undo_scaling(predictions, X_test, scaler_object, lstm=True)
    unscaled_df = predict_df(unscaled, original_df)
    
    get_scores(unscaled_df, original_df, 'LSTM')
    
    plot_results(unscaled_df, original_df, 'LSTM')

In [ ]:
# lstm_model(train, test)

In [ ]:
pickle.dump(model_scores, open( "model_scores.p", "wb" ) )